# Modelo realizado con *embeddings* para o TFM **"Estimación automática de signos de depresión a partir de análises de texto."** do Máster universitario en tecnoloxías de análise de datos masivos: Big Data no curso académico 2019/2020

## Autor: Manuel Ramón Varela López

- Ver **Readme** para as instruccións de uso. 

### Pasos previos

- **Comezamos cos datos de configuración do notebook.** 
    - Indicamos o ficherio JSON cas preguntas e respostas do test BDI.
    - Indicamos o directorio onde se gardan os documentos XML cas publicacións dos usuarios.
    - Indicamos o ficheiro que contén os resultados reais.

In [ ]:
questions_file = "inquerito.json"
dir_corpus = 'corpus'
file_real_results = 'Depression Questionnaires_anon.txt'

- **Improtamos as librerías**

In [ ]:
%matplotlib inline
import json
import xml.etree.ElementTree as ET
import os
import pandas as pd
import math
import numpy as np
import shutil
import matplotlib.pyplot as plt
import gensim
import nltk
import gensim.downloader as api

- **Configuración**

In [ ]:
pd.options.display.max_columns = None

### Comezo do script

- **Cargamos as preguntas.** 
    - Cargamos as preguntas do test BDI do arquivo indicado ao comezo do script.

In [ ]:
with open(questions_file) as json_file:
    questions = json.load(json_file)

- **Consultamos o número de xml cás publicacións de Reddit**.
    - Collemos todos os arquivos XML (un para cada usuario) no directorio na que se gardan estes arquivos indicado ao comezo do script.

In [ ]:
corpus_files = [f for f in os.listdir(dir_corpus) if os.path.isfile(os.path.join(dir_corpus, f))]

- **Lemos os arquivos XML**.
    - Preocesamos esos arquivos e gardámola información toda nun dict.

In [ ]:
data = []
for file in corpus_files:
    dataElement = {}
    writings = []
    tree = ET.parse(dir_corpus + os.path.sep + file)
    root = tree.getroot()
    for child in root:
        if child.tag == 'ID':
            dataElement['id']=child.text
        elif child.tag=='WRITING':
            writing = {}
            for wriIter in child:
                if wriIter.tag == 'TITLE':
                    writing['title']=wriIter.text
                elif wriIter.tag == 'DATE':
                    writing['date']=wriIter.text
                elif wriIter.tag == 'INFO':
                    writing['info']=wriIter.text
                elif wriIter.tag == 'TEXT':
                    writing['text']=wriIter.text
            writings.append(writing)
    dataElement['corpus'] = writings
    data.append(dataElement)

- **Cargamos o modelo pre-adestrado**.
    - Se non está dispoñible, descárgase.

In [ ]:
#Con este modelo
model = api.load("word2vec-google-news-300")

- **Procesamos o corpus de cada usuario**
    - Convertimos o texto e o título das publicacións dos usuarios en arrays de palabras.
    - Eliminamos as stop-words.

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')
for user in data:
    corpus = user.get("corpus")
    for document in corpus:
        #Quitamos as stop words do titulo
        title = document.get("title")
        document['title'] = [word for word in gensim.utils.simple_preprocess(str(title)) if word not in stop_words]
        text = document.get("text")
        document['text'] = [word for word in gensim.utils.simple_preprocess(str(text)) if word not in stop_words]

- **Construimos o array de palabras co que se calculará a similitude co corpus do usuario para cada pregunta**
    - Recibimos un texto cas palabras que se van calcular as similitudes.
    - Devolvemos un array de palabras eliminando as stopWords.

In [ ]:
def build_query(texto):
    q_aux = [word for word in gensim.utils.simple_preprocess(texto) if word not in stop_words]
    return q_aux

- **Calculo de similitude entre dúas palabras**
    - Calculamos a similitude entre dúas palabras.
    - Cada palabra é representada por un vector.
    - A similitude é o coseno entre os vectores de cada palabra.
    - Se algunha das palabras non está no modelo, devolvemos **None**.

In [ ]:
def cosSim(word1,word2):
    if (word1 in model) and (word2 in model):
        return model.similarity(word1, word2)
    return None    

- **Creamos un DataFrame para gardar os resultados.**

In [ ]:
#Recorremos todas as preguntas
d = ['subject']
for question in questions:
    d.append(question['question_number'])

#Creamos os dataframes para as medidas
results = pd.DataFrame(columns=d)
#for i in range(medidas):
    #aux = 
    #results.append(aux)

- **Realizamos o cálculo das similitudes entre pregunta e resposta máis alta e o corpus do usuario**
    - Para cada palabra do array cas palabras de cada pregunta e a resposta máis alta calculamos a similitude con cada palabra do corpus do usuario.
    - Unha vez que teñamos todas as similitudes, calculamos a medida de similitude de todas as similitudes.

In [ ]:
#Recorremos todos os usuarios
for user in data:
    
    #Collemos o id do usuario
    subject = user['id']
    
    #Imos gardando as medidas para cada usuario
    subject_res = {'subject':subject}
    #subject_res.append()
    
    #Collemos as preguntas
    for question in questions:
        #print(question)
        
        #Collemos o texto da pregunta e o numero
        question_text = question['question_text']
        question_number = question['question_number']
        
        #Collemos a ultima resposta
        answers = question['answers']
        answer = answers[len(answers)-1]
        answer_text = answer['answer_text']
        
        #Collemos cada palabra da pregunta
        question_array = build_query(question_text + " " + answer_text)
        simCoseno = []
        for word_question in question_array:
            
            #Collemos o corpus do usuario
            corpus = user.get("corpus")
            
            #Recorremos todas os textos das palabras
            for coment in corpus:
                
                #Recorremos todas as palabras do titulo
                for word_title in coment.get('title'):
                    
                    simCos = cosSim(word_question,word_title)
                    if simCos is not None:
                        simCoseno.append(simCos)   
                
                #Recorremos todas as palabras do text
                for word_text in coment.get('text'):

                    simCos = cosSim(word_question,word_text)
                    if simCos is not None:
                        simCoseno.append(simCos)
        
        #Tras calcular todas as similirades entre palabras da pregunta e dos textos calculamos o medio
        score = np.mean(simCoseno)
        subject_res[question_number] = score
        
    #Gardamos os scores para cada usuario
    results = results.append(subject_res,ignore_index=True)
    
    #Indicamos que un usuario rematou de ler
    print("User " + subject + " finished.")

- **Mostramos os resultados obtidos**
    - Indicamos que a columna "subject" é índice do DataFrame.
    - Mostramos o data frame, para cada usuario e pregunta indicamos a similitude entre a pregunta e a resposta máis alta e o corpus do usuario.

In [ ]:
results=results.set_index('subject')
display(results)

- **Normalización**
    - Normalizamos cada resposta a cada pregunta de cada usuario a un valor entre 0 e 1.
    - Despois multiplicamos polo valor do número de preguntas que ten a resposta menos 1.
    - Calculamos o floor para obter un número enteiro.
    - Ese valor e a resposta da pregunta. Terá que estar entre 0 e 3 para as preguntas con 4 respostas e 0 e 6 para as preguntas de 7 respostas.

In [ ]:
for question in questions:
    q = question['question_number']
    cats = len(question['answers'])

    #Results1
    if (max(results[q])-min(results[q])) > 0:
        results[q] = np.floor((results[q]-min(results[q]))/(max(results[q])-min(results[q]))*(cats-1))
    results[q] = pd.to_numeric(results[q],downcast='integer')

- **Cambiamos a numeración das preguntas que teñen 7 respostas**

In [ ]:
for aux2 in [16,18]:
    results.loc[results[aux2] == 1,aux2] = '1a'
    results.loc[results[aux2] == 2,aux2] = '1b'
    results.loc[results[aux2] == 3,aux2] = '2a'
    results.loc[results[aux2] == 4,aux2] = '2b'
    results.loc[results[aux2] == 5,aux2] = '3a'
    results.loc[results[aux2] == 6,aux2] = '3b'

- **Mostramos as respostas de cada usuario para cada unha das preguntas.**

In [ ]:
display(results)

### Avaliación

- **Cargamos os datos reais e gardámolos nun DataFrame**

In [ ]:
real_results = pd.read_csv(file_real_results,index_col=False,header=None,sep='\t')
real_results.columns = d
real_results=real_results.set_index('subject')

- **Mostramos os resultados reais**

In [ ]:
display(real_results)

- **Funcións necesarias para o calculo dos resultados**

In [ ]:
#Necesitamos esta función porque algunhas preguntas teñen letra
def transform(answer):
    if(type(answer)!=str):
        return answer
    elif len(answer) == 1:
        return int(answer)
    elif len(answer) == 2:
        return int(answer[0])

def dhcl_score(real,estimed):
    if (real <= 9) and (estimed<=9):
        return 1
    elif (real>29) and (estimed>29):
        return 1
    elif (real>9) and (real<=18) and (estimed>9) and (estimed<=18):
        return 1
    elif (real>18) and (real<=29) and (estimed>18) and (estimed<=29):
        return 1
    else:
        return 0

- **Calculamos as 4 medidas de avaliación descritas no TFM**

In [ ]:
mad = 3
score_array = []

#Recorremos todos os usuarios
for identificador in real_results.index.values:
    hits = 0
    crs = 0
    scr_real = 0
    scr_stm = 0
    #Recorremos todas as preguntas
    for question in questions:
        q = question['question_number']
        real_a = real_results.loc[identificador,q]
        estimated_a = results.loc[identificador,q]

        #Contamos as pregutnas acertadas
        if real_a == estimated_a:
            hits = hits + 1

        #Contamos camo de cerca estamos
        crs_aux = (mad - abs(transform(real_a)-transform(estimated_a)))/mad
        crs = crs + crs_aux

        #Calculamos os valores de depresion
        scr_real = scr_real + transform(real_a)
        scr_stm = scr_stm + transform(estimated_a)

    #Calculamos o porcentaxe de preguntas acertadas
    hit_score_aux = hits / len(questions)
    cls_score_aux = crs / len(questions)
    dl = (63 - abs(scr_real - scr_stm))/63
    dhcl = dhcl_score(scr_real,scr_stm)
    score_array.append({'subject':identificador,
                        'hit rate score':hit_score_aux,
                        'closeness rate score':cls_score_aux,
                        'real score':scr_real,
                        'estimated_score':scr_stm,
                        'dl':dl,
                        'dchr':dhcl})


score = pd.DataFrame(score_array)
score = score.set_index('subject')

- **Mostramos as 4 medidas de avaliación para cada un dos suxeitos**

In [ ]:
display(score)

- **Mostramos as medidas de avaliación do modelo proposto**

In [ ]:
display(score[['hit rate score','closeness rate score','dl','dchr']].describe())